# Calculating handicaps for WOBYC dinghy racing
This notebook was originally created for Waveney and Oulton Broad Yacht Club to demonstrate the calculation of a personalised handicap for the Oulton Rater XX2, for use in club dinghy racing. The data for XX2 is used with permission, and all other personal data has been redacted to comply with GDPR.

In a sailing race a boat's finish time $t$ is modified by the handicap $PY$ (also known as the PY number)  to give the corrected time $T_\textrm{corrected} = \frac{1000\, T}{PY}$. Dinghies generally have values of around $PY \approx 1000$, with larger handicaps corresponding to slower boats.

To compute a personalised handicap for the boat 'XX2', we assume that in each race it should attain a corrected time equal to the median corrected time. This allows us to compute a personalised handicap for each race
\begin{equation}
PY_r = \frac{1000 \, T_\textrm{XX2}}{\textrm{median}(T_\textrm{corrected})}.
\end{equation}
The value of $PY_r$ will vary significantly between races depending on the performance of 'XX2', but a good estimate of the true handicap can be obtained by taking the median of the personalised handicap of each individual race
\begin{equation}
PY_\textrm{XX2} \approx \textrm{median}(PY_r)
\end{equation}
Note that the median is used here to ensure outliers do not skew the handicap; this will be discussed further on.


We start by loading the necessary Python packages:

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
sns.set(style='darkgrid')
%matplotlib inline

The main purpose of the code is to compute a handicap for Oulton Rater XX2, but the code has been generalised to allow for any competitor by changing the value of `target_sailnumber`:

In [ ]:
target_sailnumber = 'XX2'

The function `anonymise` is used to redact personal data from a pandas `DataFrame` before displaying to comply with GDPR:

In [ ]:
def anonymise(df):
    df2 = df.copy()
    for col in ['HelmName', 'CrewName', 'SailNo', 'Boat', 'Class', 'PY', 'Elapsed', 'Corrected']:
        if col in df.columns:
            df2[col] = 'redacted (' + type(df2[col].iloc[0]).__name__ + ')'
    return df2

We're going to want to download some race data from the WOBYC website. We can use the requests package along with BeautifulSoup to parse Sailwave race tables from a chosen URL, which can then be loaded into a Pandas `DataFrame`:

In [ ]:
url = 'http://redacted/../redacted.htm'
url = 'http://wobyc.com/wp-content/uploads/2020/08/Saturday-Fast-Handicap-August.htm'
soup = BeautifulSoup(requests.get(url).text, "lxml")
races = soup.find_all('table', attrs={'class': 'racetable'})
races = [str(race) for race in races]
anonymise(pd.read_html(races[0])[0].head())

We can then proceed to clean up the data. Note that we recompute the elapsed time from the corrected time instead of using the `'Elapsed'` column in the original data; this is to avoid problems with average lap races.

In [ ]:
df = pd.read_html(races[0])[0].drop(['HelmName', 'CrewName', 'Boat'], axis=1)
df.set_index('Rank', inplace=True)
df = df[['Class', 'SailNo', 'PY', 'Corrected']]
df.dropna(inplace=True)
df['SailNo'] = df['SailNo'].astype('str')
df['Class'] = df['Class'].astype('string')
df['PY'] = df['PY'].astype('int')
df['Corrected'] = pd.to_timedelta(df['Corrected']).dt.total_seconds()
df['Elapsed'] = df['Corrected'] / df['PY'] * 1000
anonymise(df.head())

We define the function `sailwave_to_df` to automatically load an html race table into a `DataFrame` and clean up the data:

In [ ]:
def sailwave_to_df(race, year):
    try:
        """Converts a Sailwave table in html format to a pandas dataframe"""
        df = pd.read_html(race, index_col='Rank')[0]
        df = df[['Class', 'SailNo', 'PY', 'Corrected']]
        df.dropna(inplace=True)
        df['SailNo'] = df['SailNo'].astype('string')
        df['Class'] = df['Class'].astype('string')
        df['PY'] = df['PY'].astype('int')
        df['Corrected'] = pd.to_timedelta(df['Corrected']).dt.total_seconds()
        df['Elapsed'] = df['Corrected'] * df['PY'] / 1000
        df['Year'] = year
        return df
    except KeyError:
        raise

We also define the function `scrape_sailwave` which scrapes a given URL for sailwave race tables and returns a list of races in `DataFrame` format using `sailwave_to_df`:

In [ ]:
def scrape_sailwave(url, year):
    race_list = []
    for link in BeautifulSoup(requests.get(url).text, "lxml").find_all('a'):
        anchor = link.attrs["href"] if "href" in link.attrs else ''
        if anchor.endswith('htm'):
            soup = BeautifulSoup(requests.get(anchor).text, "lxml")
            tables = soup.find_all('table', attrs={'class': 'racetable'})
            for race in tables:
                try:
                    temp = sailwave_to_df(str(race), year)
                    race_list.append(temp)
                except KeyError:
                    pass
    return race_list

We can then use `scrape_sailwave` on a list of predefined URLs to obtain a list of races:

In [ ]:
URLs = [['http://redacted/../redacted1.htm', 2020],
        ['http://redacted/../redacted2.htm', 2020],
        ['http://redacted/../redacted3.htm', 2020]]
URLs = [['http://wobyc.com/racing-results-winter-2019-2020/', 2020],
        ['http://wobyc.com/racing-results-summer-2020/', 2020],
        ['http://wobyc.com/oulton-week-2020/', 2020]]
race_tables = []
for [url, year] in URLs:
    race_tables.extend(scrape_sailwave(url, year))
anonymise(race_tables[0].head())

For each race which the target boat sailed, a handicap is computed such that the XX2's corrected time is equal to the median corrected time for that race. Note that we use the median and not the mean; this is to ensure that the handicap is not disproportionately sensitive to outliers. 

In [ ]:
handicaps_df = []
for df in race_tables:
    if target_sailnumber in df['SailNo'].values:
        df_target = df.query('SailNo == @target_sailnumber')
        df_remaining = df.query('SailNo != @target_sailnumber')
        median_time = df_remaining['Corrected'].median()
        target_time = df_target['Elapsed'].iloc[0]
        handicap = 1000 * target_time / median_time
        year = df['Year'].iloc[0]
        handicaps_df.append([year, handicap])
handicaps_df = pd.DataFrame(handicaps_df, columns=['Year', 'Handicap'])
handicaps_df.head()

Full race data has only been uploaded to the website since the start of 2020. However, for XX2 we can supplement these results with a pre-prepared Excel spreadsheet containing additional data from 2018 and 2019:

In [ ]:
if target_sailnumber == 'XX2':
    try:
        xl = pd.ExcelFile('Catastrophe-Handicap.xlsx')
        sheets = (xl.parse(name) for name in xl.sheet_names if name != 'Sheet1')
        excel_data = (
            [df['Elapsed.1'].loc[0] / (df.eval('Elapsed / Handicap').median()),
             int(df['Year'].loc[0])] for df in sheets)
        excel_df = pd.DataFrame(excel_data, columns=['Handicap', 'Year'])
        handicaps_df = handicaps_df.append(excel_df, ignore_index=True)
    except Exception:
        print('Failed to load Excel file.')

We can then view some summary statistics about the estimated handicaps:

In [ ]:
handicaps_df['Handicap'].describe().round().astype(int)

Note that the mean is 12 points higher than the median. This is due to the mean's sensitivity to outliers; these outliers will be swewed towards higher handicaps for races where the target boat has performed particularly badly, perhaps due to a capsize or technical problems.

To get a better idea of the distribution of handicaps we plot a boxplot of the data:

In [ ]:
median_handicap = round(handicaps_df['Handicap'].median())
fig1 = plt.figure()
fig1.set_size_inches(3, 8)
ax1 = sns.boxplot(y='Handicap', data=handicaps_df, showfliers=False)
plot_title = 'Sail Number: ' + target_sailnumber + '\n Handicap: ' + str(median_handicap)
ax1.set(title=plot_title)
ax1.yaxis.set_major_locator(ticker.MultipleLocator(10))

We see that there's a fairly substantial variation between the upper and lower quartiles, indicating some uncertainty in the true handicap. Some of this variability can be explained by grouping the handicaps by year:

In [ ]:
handicaps_by_year = handicaps_df.groupby('Year')['Handicap']
handicaps_by_year.describe().round().astype(int)

In [ ]:
fig2 = plt.figure()
fig2.set_size_inches(6, 10)
ax2 = sns.boxplot(x='Year', y='Handicap', data=handicaps_df, showfliers=False)
median_handicaps = handicaps_by_year.median().round().astype(int)
plot_title = 'Sail Number: ' + target_sailnumber + '\n Handicaps: '
plot_title += ', '.join((str(median_handicaps.loc[i]) + ' (' + str(i) + ')'
                        for i in median_handicaps.index))
ax2.set(title=plot_title)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(10))

From the grouped boxplots we observe that the handicap has undergone a drastic drop since 2018. 2018 was the first year the current owners sailed the boat, so it is reasonable to assume this drop in handicap corresponds to an increase in skill level as they've gained experience sailing it. We also observe that the handicap has much less variability in 2020, which increases our confidence in the estimated handicap.

It is important to consider the sample sizes for each year. The number of races used in the computation for each year is given below:

In [ ]:
handicaps_by_year.count()

We observe that 2019 has significantly more races than 2020, and we can thus expect the 2019 handicap to be a more accurate prediction of the true handicap for that year. To ensure the handicap does not vary too significantly from year to year, we take the final handicap as the median handicap from the two most recent years, in this case 2019 and 2020:

In [ ]:
final_handicap = round(handicaps_df.query('Year >= 2019')['Handicap'].median())
print('The estimated handicap is', final_handicap)